In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
data = pd.read_csv('data.txt', header = None, sep = ";")
data = data[data[1]=='Johann Sebastian Bach']
data = data.reset_index(drop=True)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data[2] = data[2].str.strip('[]')
data[2] = data[2].str.split(',')
data[2] = data[2].apply(np.array)
data[3] = data[2].apply(set)
total_set = set.union(*data[3])
pitchnames = sorted(set(item for item in total_set))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
seq_len = 100

In [5]:
all_notes = np.concatenate(data[2])
note_counts = np.unique(all_notes, return_counts = True)[1] # Sorted by default
weights = 1 / note_counts
weights = weights / np.sum(weights)
weights = -1/np.log(weights)
weights = torch.Tensor(weights).to(device)

In [6]:
def get_inp_out(data):
    network_input_np = []
    network_output_np = []

    for j in tqdm(range(data.shape[0])):
        for i in range(0, len(data[2][j]) - seq_len, 1):
            sequence_in = data[2][j][i:i + seq_len]
            sequence_out = data[2][j][i + seq_len]
            network_input_np.append([note_to_int[char] for char in sequence_in])
            network_output_np.append(note_to_int[sequence_out])
    return network_input_np, network_output_np

In [7]:
data_in, data_out = get_inp_out(data)

100%|██████████| 89/89 [00:24<00:00,  3.64it/s]


In [8]:
batch_size = 128
data_in = torch.Tensor(data_in)
data_out = torch.Tensor(data_out)
dataset = TensorDataset(data_in, data_out)
train_dataset, val_dataset = random_split(dataset, [int(np.ceil(len(data_in)*0.99)), int(np.floor(len(data_in)*0.01))])
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [9]:
class DigitClassifierNetwork1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.conv5 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3)
        self.fc1 = nn.Linear(32, len(pitchnames))
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = F.relu(self.conv5(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = x.view(-1, 32)
        x = F.dropout(x,p =0.2,training = True)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)
    


In [10]:
model = DigitClassifierNetwork1().to(device)

In [11]:
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
criterion = nn.CrossEntropyLoss(weight = weights)
num_epochs = 100

19.33

In [12]:
count = 0
for epoch in range(num_epochs):
    for i, (batch_x, batch_y) in enumerate(train_data_loader):
        # Put data in the correct device
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).long()
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(batch_x.view(batch_size,1,seq_len))
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, batch_y)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        count += 1

        #if count % 5000 == 0:
    correct = 0
    total = 0
    v_loss = 0

    for val_x, val_y in val_data_loader:

        # Put data in the correct device
        val_x = val_x.to(device)
        val_y = val_y.to(device).long()
        # Forward pass only to get logits/output
        with torch.no_grad():
            output = model(val_x.view(batch_size,1, seq_len))

        # Get predictions from the maximum value
        _, predicted = torch.max(output, 1)
        val_bloss = criterion(output, val_y)
        v_loss += val_bloss*batch_size

        # Total correct predictions
        total += batch_size
        correct += (predicted == val_y).sum()
    accuracy = 100 * correct / total
    v_loss = v_loss/total

    # Print Loss
    print('Epoch: {}. Loss: {}. ValLoss: {}. Accuracy: {} %'.format(epoch, loss.item(), v_loss, accuracy))

Epoch: 0. Loss: 4.03707218170166. ValLoss: 4.284275054931641. Accuracy: 4 %
Epoch: 1. Loss: 4.128210544586182. ValLoss: 4.243627071380615. Accuracy: 4 %
Epoch: 2. Loss: 4.281319618225098. ValLoss: 4.235813140869141. Accuracy: 4 %
Epoch: 3. Loss: 4.379147529602051. ValLoss: 4.231405258178711. Accuracy: 3 %
Epoch: 4. Loss: 4.216419219970703. ValLoss: 4.237486839294434. Accuracy: 4 %
Epoch: 5. Loss: 4.376975059509277. ValLoss: 4.220913887023926. Accuracy: 4 %
Epoch: 6. Loss: 4.326277256011963. ValLoss: 4.221240520477295. Accuracy: 4 %
Epoch: 7. Loss: 4.323661804199219. ValLoss: 4.224662780761719. Accuracy: 3 %
Epoch: 8. Loss: 4.0985426902771. ValLoss: 4.220141410827637. Accuracy: 3 %
Epoch: 9. Loss: 4.242547035217285. ValLoss: 4.217456340789795. Accuracy: 3 %
Epoch: 10. Loss: 4.198811054229736. ValLoss: 4.216111660003662. Accuracy: 4 %
Epoch: 11. Loss: 4.266030311584473. ValLoss: 4.227846145629883. Accuracy: 4 %
Epoch: 12. Loss: 4.290412425994873. ValLoss: 4.215940952301025. Accuracy: 4 %

In [13]:
torch.save(model.state_dict(), 'model_1')

In [29]:
pred_len = 200
count = 0
seq = [note_to_int[note] for note in data[2][0][:100]]
seq = torch.Tensor(seq).view(1,1,100).to(device)
prediction = []
for i in range(pred_len):
    count+=1
        new_note = model(seq)
    _, new_note = torch.max(new_note, 1)
    seq = torch.cat((seq, new_note.view(1,1,1).float()), 2)[:, :, 1:]
    new_note = new_note.cpu().numpy()
    prediction.append(new_note)

count 1
torch.Size([1, 1, 100])
count 2
torch.Size([1, 1, 100])
count 3
torch.Size([1, 1, 100])
count 4
torch.Size([1, 1, 100])
count 5
torch.Size([1, 1, 100])
count 6
torch.Size([1, 1, 100])
count 7
torch.Size([1, 1, 100])
count 8
torch.Size([1, 1, 100])
count 9
torch.Size([1, 1, 100])
count 10
torch.Size([1, 1, 100])
count 11
torch.Size([1, 1, 100])
count 12
torch.Size([1, 1, 100])
count 13
torch.Size([1, 1, 100])
count 14
torch.Size([1, 1, 100])
count 15
torch.Size([1, 1, 100])
count 16
torch.Size([1, 1, 100])
count 17
torch.Size([1, 1, 100])
count 18
torch.Size([1, 1, 100])
count 19
torch.Size([1, 1, 100])
count 20
torch.Size([1, 1, 100])
count 21
torch.Size([1, 1, 100])
count 22
torch.Size([1, 1, 100])
count 23
torch.Size([1, 1, 100])
count 24
torch.Size([1, 1, 100])
count 25
torch.Size([1, 1, 100])
count 26
torch.Size([1, 1, 100])
count 27
torch.Size([1, 1, 100])
count 28
torch.Size([1, 1, 100])
count 29
torch.Size([1, 1, 100])
count 30
torch.Size([1, 1, 100])
count 31
torch.Size

In [30]:
np.savetxt('train.txt', prediction)

In [34]:
pred_len = 200
seq1 = val_dataset[2][0]
seq = seq1.view(1,1,100).to(device)
prediction = []
for i in range(pred_len):
    with torch.no_grad():
        new_note = model(seq)
    _, new_note = torch.max(new_note, 1)
    seq = torch.cat((seq, new_note.view(1,1,1).float()), 2)[:, :, 1:]
    new_note = new_note.cpu().numpy()
    prediction.append(new_note)

In [35]:
np.savetxt('val.txt', prediction)

In [19]:
predicted_train = np.loadtxt('train.txt')
predicted_val = np.loadtxt('val.txt')
predicted_train = [int_to_note[note] for note in predicted_train]
predicted_val = [int_to_note[note] for note in predicted_val]

In [23]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in predicted_val:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [24]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "val_2.mid")

'val_2.mid'